# Sleep Health and Lifestyle Analysis

Este notebook tiene como objetivos:
1. **Identificación de Arquetipos**: Clustering no supervisado usando K-Means.
2. **Predicción de Enfermedades**: Clasificación supervisada usando KNN para predecir 'Sleep Disorder'.

## 1. Carga y Limpieza de Datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Configuración de gráficos
sns.set(style="whitegrid")

# Cargar el dataset
df = pd.read_csv('datasets/Sleep_health_and_lifestyle_dataset.csv')
df.head()

In [ ]:
# Rellena los valores nulos en 'Sleep Disorder' con 'None'
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('None')

# Normaliza 'BMI Category'
df['BMI Category'] = df['BMI Category'].replace('Normal Weight', 'Normal')

# Separar 'Blood Pressure' en 'BP_Systolic' y 'BP_Diastolic'
df[['BP_Systolic', 'BP_Diastolic']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)
df = df.drop(columns=['Blood Pressure'])

# Eliminar 'Person ID'
df = df.drop(columns=['Person ID'])

# Verificar info del dataframe limpio
df.info()

## 2. Preprocesamiento de Datos
Para algoritmos basados en distancia como K-Means y KNN, es crítico escalar las variables numéricas y codificar las categóricas.

In [ ]:
# Definir columnas
numeric_cols = ['Age', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level', 
                'Stress Level', 'Heart Rate', 'Daily Steps', 'BP_Systolic', 'BP_Diastolic']
categorical_cols = ['Gender', 'Occupation', 'BMI Category']

# Escalar numéricas
scaler = StandardScaler()
X_num = pd.DataFrame(scaler.fit_transform(df[numeric_cols]), columns=numeric_cols)

# OneHotEncoding para categóricas
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_cat = pd.DataFrame(encoder.fit_transform(df[categorical_cols]), 
                     columns=encoder.get_feature_names_out(categorical_cols))

# Combinar en X_processed
X_processed = pd.concat([X_num, X_cat], axis=1)

print("Shape de X_processed:", X_processed.shape)
X_processed.head()

## 3. Análisis de Arquetipos (clustering K-Means)
Usaremos el método del codo para encontrar un k óptimo y luego interpretaremos los clusters.

In [ ]:
# Método del codo
inertias = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_processed)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(8, 4))
plt.plot(K_range, inertias, 'bo-')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo para K Óptimo')
plt.show()

In [ ]:
# Entrenar con k=3 (Ejemplo razonable según el problema)
kmeans_final = KMeans(n_clusters=3, random_state=42, n_init=10)
df['Archetype'] = kmeans_final.fit_predict(X_processed)

# Tabla resumen para interpretación
archetype_profile = df.groupby('Archetype')[numeric_cols].mean().round(2)
print("Perfil Promedio por Arquetipo:")
display(archetype_profile)

# Conteo de personas por arquetipo
print("\nDistribución de Arquetipos:")
print(df['Archetype'].value_counts())

### Visualización de Clusters (PCA)

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_processed)

plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=df['Archetype'], palette='viridis', s=100)
plt.title('Visualización de Arquetipos con PCA (2D)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()

## 4. Clasificación de Enfermedades (KNN)
Predeciremos la columna 'Sleep Disorder' ('None', 'Sleep Apnea', 'Insomnia').

In [ ]:
# Definir X e y
X = X_processed
y = df['Sleep Disorder']

# Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predicciones
y_pred = knn.predict(X_test)

# Evaluación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=knn.classes_, yticklabels=knn.classes_)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - KNN')
plt.show()